In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np

In [18]:
epochs=4
batch_size=4
lr=0.0001

transform=transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])

train_data=torchvision.datasets.CIFAR10(root='./data',train=True,transform=transform,download=True)
test_data=torchvision.datasets.CIFAR10(root='./data',train=False,transform=transform,download=True)

trainloader=torch.utils.data.DataLoader(dataset=train_data,batch_size=batch_size,shuffle=True)
testloader=torch.utils.data.DataLoader(dataset=test_data,batch_size=batch_size,shuffle=False)

Files already downloaded and verified
Files already downloaded and verified


In [19]:
classes=('plane','car','bird','cat','deer','dog','frog','horse','sheep','truck')

class Convnet(nn.Module):
  def __init__(self):
    super(Convnet,self).__init__()
    self.conv1=nn.Conv2d(3,6,5)
    self.pool=nn.MaxPool2d(2,2)
    self.conv2=nn.Conv2d(6,16,5)
    self.fc1=nn.Linear(16*5*5,120)
    self.fc2=nn.Linear(120,84)
    self.fc3=nn.Linear(84,10)
  def forward(self,x):
    x=self.pool(F.relu(self.conv1(x)))
    x=self.pool(F.relu(self.conv2(x)))
    x=x.view(-1,16*5*5)
    x=F.relu(self.fc1(x))
    x=F.relu(self.fc2(x))
    x=(self.fc3(x))
    return x

model=Convnet()
loss=nn.CrossEntropyLoss()
optimize=torch.optim.SGD(model.parameters(),lr=lr)

n_steps=len(trainloader)

for epoch in range(epochs):
  print(epoch)
  for i,(images,ans) in enumerate(trainloader):
    op=model(images)
    l=loss(op,ans)
    optimize.zero_grad()
    l.backward()
    optimize.step()



0
1
2
3


In [20]:
with torch.no_grad():
  n_corr=0
  n_samples=0
  for (images,labels) in testloader:
    op=model(images)
    _,pred=torch.max(op,1)
    n_samples+=labels.size(0)
    n_corr+=(pred==labels).sum().item()

print(n_corr/n_samples)

0.1522
